In [1]:
from spotipy.oauth2 import SpotifyOAuth
from sqlalchemy import create_engine
import pandas as pd
import sqlalchemy
import sqlite3
import spotipy
import math

## 1. Extract

In [2]:
scope = "user-library-read"
username = "Hunor Tot-Bagi"
client_id = "ff5d45edced9494aaee5427d666e5a13"
client_secret = "5123bcdbaeac4450a24889181a2960b2"
redirect_uri = "http://localhost:8000/callback"

sp = spotipy.Spotify(auth_manager=SpotifyOAuth(scope=scope, username=username, client_id=client_id, client_secret=client_secret, redirect_uri=redirect_uri))

data = sp.current_user_saved_tracks(limit=50)

In [3]:
# Creating empty lists to store the data
song_names = []
artist_names = []
song_duration = []
popularity = []
release_date = []
added_at = []

# Loop iterates through the 'items' in the 'data' and appends the name of the song, name of the artist, and song duration
for song in data['items']:
    song_names.append(song['track']['name'])
    artist_names.append(song['track']['album']['artists'][0]['name'])
    song_duration.append(song['track']['duration_ms'])
    popularity.append(song['track']['popularity'])
    release_date.append(song['track']['album']['release_date'])
    added_at.append(song['added_at'])
    
# Creates a dictionary using the three lists created earlier and assigns each list to a key in the dictionary
song_dict = {
    "song_name" : song_names,
    "artist_name" : artist_names,
    "song_duration" : song_duration,
    "popularity" : popularity,
    "release_date" : release_date,
    "added_at" : added_at
}

# Converts the dictionary to a Pandas DataFrame with columns for the song name, artist name, and song duration
song_df = pd.DataFrame(song_dict, columns = ['song_name',
                                             'artist_name',
                                             'song_duration', 
                                             'popularity',
                                             'release_date',
                                             'added_at'])

# Renames the column headers of the DataFrame to be more readable
song_df = song_df.rename(columns={'song_name': 'Song name',
                                  'artist_name': 'Artist name',
                                  'song_duration': 'Miliseconds',
                                  'popularity': 'Popularity',
                                  'release_date': 'Song release date',
                                  'added_at' : 'Date added to playlist'})

song_df.head(5)

Song name     Artist name  Miliseconds  Popularity  \
0          jealousy, jealousy  Olivia Rodrigo       173160          83   
1   Unholy (feat. Kim Petras)       Sam Smith       156943          92   
2        Prođi Sa Mnom Bosnom            Amna       171432          53   
3                       Etida           Edita       193120          38   
4  Killers From The Northside        Kordhell       187500          68   

  Song release date Date added to playlist  
0        2021-05-21   2023-04-29T06:17:31Z  
1        2022-09-22   2022-12-31T16:41:05Z  
2        2022-10-23   2022-10-30T15:04:02Z  
3        2022-09-23   2022-09-26T22:29:10Z  
4        2021-09-30   2022-08-26T09:04:39Z

## 2. Transform

In [4]:
# Add new columns for song duration in minutes and seconds, rounded down to the nearest integer
song_df['Minutes'] = (song_df['Miliseconds'] / 60000).apply(math.floor)
song_df['Seconds'] = ((song_df['Miliseconds'] / 1000) % 60).apply(math.floor)

In [5]:
song_df.head(5)

Song name     Artist name  Miliseconds  Popularity  \
0          jealousy, jealousy  Olivia Rodrigo       173160          83   
1   Unholy (feat. Kim Petras)       Sam Smith       156943          92   
2        Prođi Sa Mnom Bosnom            Amna       171432          53   
3                       Etida           Edita       193120          38   
4  Killers From The Northside        Kordhell       187500          68   

  Song release date Date added to playlist  Minutes  Seconds  
0        2021-05-21   2023-04-29T06:17:31Z        2       53  
1        2022-09-22   2022-12-31T16:41:05Z        2       36  
2        2022-10-23   2022-10-30T15:04:02Z        2       51  
3        2022-09-23   2022-09-26T22:29:10Z        3       13  
4        2021-09-30   2022-08-26T09:04:39Z        3        7

In [6]:
# Define a function that takes a DataFrame as input
def check_if_valid_data(df):
    
    # Check if DataFrame is empty
    if df.empty:
        print("1. No songs downloaded.")
    else:
        print('1. Data Frame is not empty.')
    
    # Primary Key Check
    # The "Date added to playlist" column is a suitable primary key candidate 
    # Since it is impossible for two songs to be added to the playlist at the exact same time
    if pd.Series(df['Date added to playlist']).is_unique:
        print('2. Primary Key column is unique.')
    else:
        raise Exception('2. Primary Key Check is violated.')
        
    # Check for Null values in DataFrame
    if df.isnull().values.any():
        raise Exception('3. Null valued found')
    else:
        print('3. No Null value found.')
        
check_if_valid_data(song_df)

1. Data Frame is not empty.
2. Primary Key column is unique.
3. No Null value found.


## 3. Load

In [7]:
# Set the location of the database file
DATABASE_LOCATION = "sqlite:///my_favorite_songs.sqlite"

# Create a database engine object using SQLAlchemy
engine = sqlalchemy.create_engine(DATABASE_LOCATION)

# Connect to the database using sqlite3
conn = sqlite3.connect('my_favorite_songs.sqlite')

# Create a cursor object to execute SQL commands
cursor = conn.cursor()

# Define an SQL query to create a table if it doesn't exist already
sql_query = """
CREATE TABLE IF NOT EXISTS my_played_tracks(
    song_names VARCHAR(200),
    artist_names VARCHAR(200),
    song_duration VARCHAR(200),
    popularity VARCHAR(200),
    release_date VARCHAR(200),
    added_at VARCHAR(200),
)
"""

In [8]:
# Try to write a pandas dataframe called 'song_df' to a SQL table called 'my_favorite_songs'
try:
    song_df.to_sql('my_favorite_songs', engine, index=False, if_exists='append')
    
# If the table already exists, print an error message
except:
    print('Data already exists in the database')
    
conn.close()
print('Closed the database successfully')

Closed the database successfully
